### 라이브러리 설치

In [1]:
pip install librosa

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install imutils

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


### 라이브러리 import

In [1]:
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import GridSearchCV
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib
from sklearn.svm import SVC
from imutils import paths

import matplotlib.pyplot as plt
import soundfile as sf
import librosa
import wave, array
import numpy as np
import pandas as pd
import pickle
import json
import sys
import os

import warnings
warnings.filterwarnings('ignore')

C:\Users\HONGHYEIN\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


### 코딩 시작

In [2]:
def get_result(y, sr):
    clf_load = joblib.load('saved_model.pkl') 
    audio_mfcc = librosa.feature.mfcc(y = y, sr = sr)
    x_test = audio_mfcc.reshape(-1)
    return clf_load.predict([x_test])[0]

In [3]:
# json 데이터 불러오기
file = 'C:/Users/HONGHYEIN/Desktop/홍혜인/4학년 2학기/캡스톤/dev/데이터/data.json'
with open(file, "r", encoding = "utf-8") as json_file:
    json_data = json.load(json_file)

# 음성 파일 불러오기
file = 'C:/Users/HONGHYEIN/Desktop/홍혜인/4학년 2학기/캡스톤/dev/데이터/call.wav'
y, sr = librosa.load(file, sr = 16000)

# 화자 수 설정하기
speakers = json_data['speakers']

In [4]:
# json 파일 사용하여 화자 분할하기
spk = []
spk_cnt = []

for i in range(0, speakers):
    spk.append([])
    spk_cnt.append(0)

for i in range(0, len(json_data['result'])):
    start_time = float(json_data['result'][i]['start_time'])
    end_time = float(json_data['result'][i]['end_time'])
    
    cutted_y = y[int(start_time)*16000:int(end_time)*16000]
    
    if json_data['result'][i]['speaker_label'] == 'spk_0':
        spk[0].append([cutted_y, sr])
        spk_cnt[0] += 1
        
    if json_data['result'][i]['speaker_label'] == 'spk_1':
        spk[1].append([cutted_y, sr])
        spk_cnt[1] += 1

In [5]:
# 화자마다 음성 파일 1초씩 자르기
# 음성 파일 테스트하기
emotion = []
total_duration = []

for i in range(0, speakers):
    emotion.append([])
    total_duration.append(0)

k = 0
while k < speakers:
    for i in range(0, spk_cnt[k]):
        duration = int(librosa.get_duration(spk[k][i][0], sr))
        total_duration[k] += duration
        for j in range(0, duration):
            cutted_y = spk[k][i][0][j*16000:(j+1)*16000]
            emotion[k].append(get_result(cutted_y, 16000))
    k += 1

In [6]:
# 테스트 결과 가공하기
emotion_count =[]

for i in range(0, speakers):
    emotion_count.append({'happiness':0, 'neutral':0, 'sadness':0, 'anger':0, 'time':total_duration[i]})

k = 0
while k < speakers:
    for i in emotion[k]:
        try: emotion_count[k][i]+= 1
        except: emotion_count[k][i] = 1
    k += 1

[{'happiness': 3, 'neutral': 0, 'sadness': 0, 'anger': 1, 'time': 4}, {'happiness': 0, 'neutral': 2, 'sadness': 0, 'anger': 0, 'time': 2}]


In [8]:
# 테스트 결과 CSV 파일로 저장하기
result_dict = {}

for i in range(0, speakers):
    spk_id = "spk_" + str(i)  
    temp_dict = {}
    temp_key = list(emotion_count[i].keys())
    temp_value = list(emotion_count[i].values())
    for j in range(0, 5):    
        temp_dict[temp_key[j]] = temp_value[j]
        result_dict[spk_id] = temp_dict

df = pd.DataFrame(result_dict).T
df.to_csv('C:/Users/HONGHYEIN/Desktop/홍혜인/4학년 2학기/캡스톤/dev/데이터/data.csv')